## 🔧 Environment Check

In [7]:
!pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.6 MB/s eta 0:00:00


In [3]:
import torch
print(torch.cuda.is_available())

False


I have a mac :\)

## 📦 Dataset Load & Preprocessing	

In [ ]:
from datasets import load_dataset
from torchvision import transforms
from PIL import Image

# Load both train and test splits
ds = load_dataset("CADCODER/GenCAD-Code", num_proc=16, split=["train", "test"], cache_dir="/Volumes/BIG-DATA/HUGGINGFACE_CACHE")
# Define image transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Preprocessing function
def preprocess(example):
    example["image"] = transform(example["image"].convert("RGB"))
    return example

# Apply transform to both splits
train_dataset = datasets[0].map(preprocess)
test_dataset = datasets[1].map(preprocess)


README.md:   0%|          | 0.00/706 [00:00<?, ?B/s]

Process SpawnPoolWorker-2:
Process SpawnPoolWorker-3:
Process SpawnPoolWorker-1:


KeyboardInterrupt: 